In [ ]:
import pandas as pd 
from os.path import join
import matplotlib.pyplot as plt
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [ ]:
data_dir = join('.','..', 'Data')
files = !ls $data_dir
files = [f for f in files if f.endswith('.csv')]
ids = list(set([x[:3] for x in files]))
# files = random.sample(files, 2)

In [ ]:
raven = pd.read_csv(join(data_dir, 'raven.csvx'))
raven = raven[:-1]
raven = raven[["Unnamed: 0", "Result"]]
raven.columns = ['Part_id', "Result"]
raven.Result = raven.Result.str.replace(",", '.').astype(np.float)
raven.set_index("Part_id", inplace=True)
raven.loc[162]

In [ ]:
results = [["Part_id", "R^2", "Intercept", "Slope", "First_block_corr", "Raven", "TAO"]]
for idx, f in enumerate(files):
    data = pd.read_csv(join(data_dir,f))
    data["Block_no"] = 0
    assert data.shape[0] == 590
    for i in list(range(11)): # add block no
        data[50*i + 40:50*(i+1) + 40].Block_no = i+1
    data = data.query("Trial_type != 'training'")
    corr = data.query("Block_no == 1").mean().Corr
    adaptive_block_correctness = data.query("Block_no > 1").groupby("Block_no").mean().Corr.to_numpy()
    block_no = np.array(range(1, 11)).reshape((-1, 1))
    model = LinearRegression(normalize=False)
    model.fit(block_no, adaptive_block_correctness)
    r_sq = round(model.score(block_no, adaptive_block_correctness), 5)
    intercept = round(model.intercept_, 5)
    slope = round(model.coef_[0], 5)
    part_id = f.split("_")[0]
    if "M" in part_id:
        raven_res = raven.loc[int(part_id.split('M')[0])] 
    elif "K" in part_id:
        raven_res = raven.loc[int(part_id.split('K')[0])]
    results.append([part_id, r_sq, intercept, slope, corr, raven_res, "TAO"])

In [ ]:
results

In [ ]:
model = LinearRegression().fit(block_no, adaptive_block_correctness)